# Library

In [1]:
import pandas as pd
import numpy as np
import re
import warnings


from nltk.tokenize import sent_tokenize
from sqlalchemy import create_engine
from tqdm import tqdm_notebook
import pickle as pk
import tensorflow as tf

from sentence_transformers import SentenceTransformer
from nltk import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# 경고무시를 위한 코드
warnings.filterwarnings('ignore')

# 데이터 로드

In [3]:
case_list_사기 = pd.read_csv('./00. Data/case_list_사기.csv')

## SBert

### 모델 로드

In [4]:
with tf.device("/device:GPU:0"):
    sbert = SentenceTransformer('jhgan/ko-sbert-sts')

In [5]:
sentence_tokened = []

for x in tqdm_notebook(case_list_사기['X']):
    sentence_tokened.append(sent_tokenize(x))

  0%|          | 0/2262 [00:00<?, ?it/s]

#### 5자 이하의 문장은 없애기

In [6]:
for cases in sentence_tokened:
  for sen in cases:
    if len(sen) <= 5:
      cases.remove(sen)

#### 한 판례가 90000자 이상을 넘어가면 해당 판례 없애기 -> 임베딩에 너무 많은 시간 소요

In [7]:
remove_cnt = 0
cases_to_remove = []
for cases in sentence_tokened:
    len_case = 0
    for sen in cases:
        len_case += len(sen)
        if len_case >= 90000:
            cases_to_remove.append(cases)
            remove_cnt += 1
            break

for cases in cases_to_remove:
    sentence_tokened.remove(cases)

print(remove_cnt)

16


In [8]:
max_len = 0
for cases in sentence_tokened:
    len_case = 0
    for sen in cases:
        len_case += len(sen)
    max_len = max(max_len, len_case)
print(max_len)
# 최대 판례 길이: 89993

89993


In [9]:
clean_sentence_tokened = []
for sent in tqdm_notebook(sentence_tokened):
    sen = []
    for s in sent:
        sen.append(re.sub('[^가-힣 ]', '', s))
    clean_sentence_tokened.append(sen)

  0%|          | 0/2246 [00:00<?, ?it/s]

In [10]:
clean_sentence_tokened[0]

['  상고이유를 판단한다',
 '불가벌적 사후행위에 대한 판단유사수신행위의 규제에 관한 법률이하 유사수신행위법이라 한다   를 위반한 행위는 그 자체가 사기행위에 해당한다거나 사기행위를 반드시 포함한다고 할 수 없고 유사수신행위법 위반죄가 형법  의 사기죄와 구성요건을 달리하는 별개의 범죄로서 서로 보호법익이 다른 이상 유사수신행위를 한 자가 출자자에게 별도의 기망행위를 하여 유사수신행위로 조달받은 자금의 전부 또는 일부를 다시 투자받는 행위는 유사수신행위법 위반죄와 다른 새로운 보호법익을 침해하는 것으로서 유사수신행위법 위반죄의 불가벌적 사후행위가 되는 것이 아니라 별죄인 사기죄를 구성한다',
 '원심은 판시와 같은 이유로 피고인의 경 피해자 공소외인에 대한 판시 사기의 점에 관한 공소사실에 대하여 유사수신행위법 에서 금지하는 유사수신행위에는 기망행위가 포함되어 있지 않고 유사수신행위법 위반죄와 사기죄는 구성요건을 달리하는 별개의 범죄로서 서로 행위의 태양이나 보호법익을 달리한다는 등의 이유로 기존 범죄인 유사수신행위법 위반죄의 가벌적 평가범위 내에 흡수되는 불가벌적 사후행위에 해당하지 않는다고 판단하였다',
 '원심판결 이유를 위 법리 및 기록에 비추어 살펴보면 이 부분 원심판단에 일부 적절하지 않은 점이 있으나 피고인의 경 피해자 공소외인에 대한 판시 사기죄가 유사수신행위법 위반죄의 불가벌적 사후행위에 해당하지 않는다고 본 원심의 판단에 불가벌적 사후행위에 관한 법리를 오해함으로써 판결에 영향을 미친 잘못이 없다',
 '나머지 상고이유에 대한 판단  이 부분 상고이유는 형사소송법  에서 정한 형보다 가벼운 형이 선고된 이 사건에서 사실오인법리오해를 내세우며 실질적으로 원심의 증거 선택 및 증명력에 관한 판단 또는 이에 기초한 사실인정을 탓하는 것이거나 원심이 인정한 사실과 다른 사실관계를 전제로 법리오해를 지적하는 취지의 주장 또는 형이 너무 무거워 부당하다는 취지의 주장에 해당하여 모두 적법한 상고이유로 볼 수 없다',
 '결론  그러므로 상고를 기각

In [11]:
embeddings_sbert = []

for sentences in tqdm_notebook(clean_sentence_tokened):
    embedding = None
    count = 0
    for sentence in sentences:
        if embedding is None:
            embedding = sbert.encode(sentence)
        else:
            embedding += sbert.encode(sentence)
        count += 1
    embeddings_sbert.append(embedding/count)

  0%|          | 0/2246 [00:00<?, ?it/s]

In [13]:
with open('./00. Data/sbert_embedding.pickle', 'wb') as fw:
    pk.dump(embeddings_sbert, fw)

# Sbert 성능 테스트

In [ ]:
with tf.device("/device:GPU:0"):
    sbert = SentenceTransformer('jhgan/ko-sbert-sts')

In [14]:
# 모델, 임베딩 로드
with open('./00. Data/sbert_embedding.pickle', 'rb') as fr:
    sbert_embedding = pk.load(fr)

### 사용자 입력 임베딩

In [17]:
from hanja_hangul.src import hanjahangul
# 사용자 입력 전처리
user_input = "저는 3년 전부터 甲에게 그 남편의 사업자금 명목으로 수 차례에 걸쳐 1,500만원을 빌려주었습니다. \
    甲은 돈을 빌릴 당시에는 남편의 건축사업이 잘되면 이자는 물론 아파트분양까지 책임지겠다고 하여 믿고 빌려주었는데, \
    이제 와서는 건축경기가 좋지 않아 파산위기에 처해 있으니 마음대로 하라고 합니다. 이 경우 甲을 사기죄로 처벌할 수 있는지요?"

user_input = hanjahangul.hanja_to_hangul_dueum(user_input)
clear_user_input = re.sub("[^가-힣. ]", '', user_input)

# 사용자 입력 문장 토큰화
user_sentence_tokened = sent_tokenize(clear_user_input)
user_sentence_tokened

['저는 년 전부터 갑에게 그 남편의 사업자금 명목으로 수 차례에 걸쳐 만원을 빌려주었습니다.',
 '갑은 돈을 빌릴 당시에는 남편의 건축사업이 잘되면 이자는 물론 아파트분양까지 책임지겠다고 하여 믿고 빌려주었는데     이제 와서는 건축경기가 좋지 않아 파산위기에 처해 있으니 마음대로 하라고 합니다.',
 '이 경우 갑을 사기죄로 처벌할 수 있는지요']

In [18]:
embedding = None
count = 0
for sentence in user_sentence_tokened:
    if embedding is None:
        embedding = sbert.encode(sentence)
    else:
        embedding += sbert.encode(sentence)
    count += 1

s_embedding = embedding / count

### 코사인 유사도 계산

In [19]:
s_similarity = cosine_similarity([s_embedding], sbert_embedding)
s_similarity = list(enumerate(s_similarity[0]))
s_similarity = sorted(s_similarity, key=lambda x : x[1], reverse = True)

In [20]:
precidents = []

for i in range(5):
    idx = s_similarity[i][0]
    sim = round(s_similarity[i][1] * 100, 2)

    precidents.append([case_list_사기['판례일련번호'][idx], case_list_사기['사건명'][idx], case_list_사기['사건종류명'][idx], case_list_사기['판례내용'][idx], sim])

In [21]:
f_result = pd.DataFrame(precidents, columns=['판례일련번호', '사건명', '사건종류명', '판례내용', '유사도'])
f_result

,판례일련번호,사건명,사건종류명,판례내용,유사도
0,185094,사기·지방교육자치에관한법률위반,형사,【피 고 인】 【상 고 인】 피고인 및 검사【변 호 인】 변호사 손지열 외 4인【원...,65.02
1,144618,특정경제범죄가중처벌등에관한법률위반(배임){피고인2에대한예비적죄명:특정경제범죄가중처벌...,형사,【피 고 인】 【항 소 인】 피고인들 및 검사【검 사】 황현덕외 1인【변 호 ...,64.64
2,150918,무고·사기미수,형사,【피 고 인】 【항 소 인】 피고인【검 사】 홍기채【변 호 인】 법무법인 지평...,64.53
3,206304,특정경제범죄가중처벌등에관한법률위반(사기)(인정된죄명:사기)·사기·업무상배임,형사,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 조홍【원심판결】 서울고...,63.74
4,226505,특정경제범죄가중처벌등에관한법률위반(횡령)[일부인정된죄명:특정경제범죄가중처벌등에관한법...,형사,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 정해원【원심판결】 서울...,63.57
